In [46]:
#загрузим основные библиотеки
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
 
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
import joblib

from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.tree import DecisionTreeRegressor

In [47]:
df_wine= pd.read_csv('data/Red.csv')
df_wine_test= pd.read_csv('data/Red_test.csv')

Задание 6.1

На основании датасета предсказания рейтинга красного вина, с которым вы работали в разделе о предобработке данных в предыдущем юните (файл Red.csv ), вам предстоит выполнить следующее:

Добавить в пайплайн, созданный вами в предыдущем юните, обработку столбца 'Region'. Кодирование 'Region' произвести с использованием OrdinalEncoder. Стандартизацию столбца 'Price' и кодирование столбца 'Country' осуществить в соответствии с примером, представленным в юните 5.

In [48]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer
# кодируем и стандартизщируем данные
ct = make_column_transformer(
    (OrdinalEncoder(), ['Region']),
    (StandardScaler(), ['Price']),
    (OneHotEncoder(), ['Country']))
print(ct)

ColumnTransformer(transformers=[('ordinalencoder', OrdinalEncoder(),
                                 ['Region']),
                                ('standardscaler', StandardScaler(), ['Price']),
                                ('onehotencoder', OneHotEncoder(),
                                 ['Country'])])


In [49]:
pipeline = Pipeline([('ct', ct), ('rf', RandomForestRegressor(random_state=42))])

In [50]:
X = df_wine[['Region','Country', 'Price']]
y = df_wine['Rating']

In [51]:
pipeline.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf', RandomForestRegressor(random_state=42))])

In [52]:
X_test = df_wine_test[['Region','Country', 'Price']]
y_test = df_wine_test['Rating']

In [53]:
# Делаем предсказания и оцениваем модель
y_pred = pipeline.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse:.4f}')

# Сохраняем пайплайн
joblib.dump(pipeline, 'pipeline_wine.pkl')


RMSE: 0.0765


d:\Python\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


['pipeline_wine.pkl']

Задание 6.2

Теперь попробуем изменить параметры случайного леса в пайплайне, полученном в предыдущем задании.

Измените параметр n_estimators в случайном лесу со значения по умолчанию до 200 , используя метод set_params.

В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округленный до четвёртого знака после запятой.

In [54]:
# изменяем параметр
pipeline.set_params(rf__n_estimators=200)

# обучаем пайплайн
pipeline.fit(X, y)

# Делаем предсказания и оцениваем модель
y_pred = pipeline.predict(X_test)

# вычисляем целевую метрику
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse:.4f}')


RMSE: 0.0761


d:\Python\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Задание 6.3

Теперь попробуем добавить стекинг в качестве модели в пайплайн.

Вам следует выполнить следующее:

Собрать StackingRegressor:
1. В качестве базовых моделей возьмите ридж-регрессию RidgeCV() и решающее дерево.
2. В качестве метамодели возьмите случайный лес с настройками (количество базовых моделей 10).
3. Все базовые модели стекинга модели должны быть с настройками по умолчанию (кроме random_state).
- Зафиксировать random_state=42 (для всех моделей).
- Заменить в пайплайне задачи 6.1 случайный лес на StackingRegressor.
- Обучить модель на тренировочной выборке.

In [55]:
# Создаем список кортежей вида: (наименование модели, модель)
estimators = [
    ('lr', RidgeCV()),
    ('dt',  DecisionTreeRegressor(random_state=42))
]

# Создаем объект класса стекинг
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10,
                                          random_state=42)
)

# формируем пайплайн со стеком
pipeline = Pipeline([('ct', ct), ('stack',reg)])

# Обучаем модель
pipeline.fit(X, y)

# Делаем предсказания и оцениваем модель
y_pred_stack = pipeline.predict(X_test)
rmse = mean_squared_error(y_test, y_pred_stack, squared=False)
print(f'RMSE: {rmse:.2f}')


RMSE: 0.18


d:\Python\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
